In [1]:
import pandas as pd
import geopandas as gpd
import getBuildings
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from shapely.geometry import LineString

import json

# Get closest

In [19]:
osm_ids = '-62640, -1070986,-1070976,-1070979,-1071007,-1070996,-1071000,-1070985,-1070974' # Bayreuth
df = getBuildings.loadBuildingsData(osm_ids)
df = df.set_geometry("way")

c:\Users\strobel\miniconda3\envs\normal\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\strobel\miniconda3\envs\normal\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\strobel\miniconda3\envs\normal\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\strobel\miniconda3\envs\normal\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The `op` parameter is deprecated and

In [52]:
data = pd.read_csv("C:/Users/strobel/Projekte/esmregio/Daten/Stadtwerke/netzkopplungspunkte/Anschluesse_Strom_Stand_02_05_2022.csv", sep=";", decimal =",", encoding="ANSI")
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data["Koordinate x"], data["Koordinate y"]), crs="EPSG:32632")
gdf = gdf.to_crs("EPSG:3857")
gdf = gdf.set_index("GIS_FID", drop=False)

In [60]:
# Very slow ~ 250min
# If I have to do this again check if gpd.sjoin_nearest works now
ids = []

multipoint = gdf.geometry.unary_union
for poly in df.geometry:
    queried_geom, nearest_geom = nearest_points(poly, multipoint)
    ids.append(gdf[gdf.geometry == nearest_geom].GIS_FID.iloc[0])
df["GIS_FID"] = ids

In [61]:
# Save
osm_fid_map = {}
for osm_id, gis_fid in zip(df.osm_id, df.GIS_FID):
    osm_fid_map[osm_id] = gis_fid
with open("OSMtoFID.json", "w") as fn:
    json.dump(osm_fid_map, fn)

# Test

In [10]:
data = pd.read_csv("C:/Users/strobel/Projekte/esmregio/Daten/Stadtwerke/netzkopplungspunkte/Anschluesse_Strom_Stand_02_05_2022.csv", sep=";", decimal =",", encoding="ANSI")
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data["Koordinate x"], data["Koordinate y"]), crs="EPSG:32632")
gdf = gdf.to_crs("EPSG:3857")
data = pd.read_csv("../Buildings-62640,-1070986,-1070976,-1070979,-1071007,-1070996,-1071000,-1070985,-1070974.csv")
buildings = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data["lon"], data["lat"]), crs="EPSG:4326").to_crs("EPSG:3857")

In [12]:
with open("OSMtoFID.json", "r") as fn:
    ma = json.load(fn)

In [15]:
buildings["GIS_FID"] = buildings.osm_id.apply(lambda x: ma[str(x)])

In [17]:
tst = buildings.set_index("GIS_FID").copy()["geometry"].to_frame()
tst["point"] = gdf.set_index("GIS_FID").geometry
tst['conn'] = tst.apply(lambda x: LineString([x["geometry"].centroid, x["point"]]), axis=1)
tst = gpd.GeoDataFrame(tst, geometry="conn")

c:\Users\strobel\miniconda3\envs\normal\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [18]:
%matplotlib qt
f, ax = plt.subplots()
buildings.plot(ax=ax)
gdf.plot(ax=ax)
tst.plot(ax=ax)

<AxesSubplot:>